In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [16]:
%run /content/drive/MyDrive/emotion_project/predict.py

모드 선택:
1. 대화 모드 (슬라이딩 윈도우)
2. 단일 문장 모드
선택 (1/2): 1
🗣️ 대화 시작! (윈도우 크기: 3, 종료: exit)
💬 입력: 치킨이 너무 맛있다
🎯 감정: 행복 (0.909), 중립 (0.084)
🎨 추천 이모지: 😄 😂 😝

💬 입력: 이 치킨 죽여주네
📖 문맥: 치킨이 너무 맛있다
🎯 감정: 행복 (0.480), 혐오 (0.310)
🎨 추천 이모지: 😄 😂 😝

💬 입력: 아주 끝내줘
📖 문맥: 치킨이 너무 맛있다 → 이 치킨 죽여주네
🎯 감정: 행복 (0.846), 중립 (0.173)
🎨 추천 이모지: 😄 😂 😝

💬 입력: 프로젝트가 잘 끝나서 기분이 좋군
📖 문맥: 이 치킨 죽여주네 → 아주 끝내줘
🎯 감정: 행복 (0.871), 중립 (0.142)
🎨 추천 이모지: 😄 😂 😝

💬 입력: 근데 좀 졸리네
📖 문맥: 아주 끝내줘 → 프로젝트가 잘 끝나서 기분이 좋군
🎯 감정: 중립 (0.730), 행복 (0.103)
🎨 추천 이모지: 😐 😑 🙂



KeyboardInterrupt: Interrupted by user

In [14]:
%run /content/drive/MyDrive/emotion_project/train.py

ModuleNotFoundError: No module named 'deep_translator'

In [4]:
!pip install deep-translator


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 4.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import os

def fix_emotion_typos(file_path, output_path=None):
    """
    한국어 연속적 대화 데이터셋의 감정 오타를 수정하는 함수

    Args:
        file_path: 원본 엑셀 파일 경로
        output_path: 수정된 파일을 저장할 경로 (None이면 _수정본 추가)
    """

    # 오타 수정 매핑
    typo_corrections = {
        "ㄴ중립": "중립",
        "ㅈ중립": "중립",
        "ㅍ": "중립",      # 'ㅍ'는 '분노'의 오타로 추정
        "분": "분노",
        "분ㄴ": "분노",
        "줄": "중립",      # '줄'은 '중립'의 오타로 추정
        "중림": "중립"
    }

    # 표준 감정 목록
    standard_emotions = ["공포", "놀람", "분노", "슬픔", "중립", "행복", "혐오"]

    print("🔍 감정 오타 수정을 시작합니다...")

    # 엑셀 파일 읽기
    try:
        df = pd.read_excel(file_path, header=1)  # 2번째 행이 헤더
        print(f"✅ 파일 로드 완료: {len(df)}개 행")
    except Exception as e:
        print(f"❌ 파일 읽기 오류: {e}")
        return

    # 감정 컬럼 확인 (3번째 컬럼이 감정)
    emotion_col = df.columns[2] if len(df.columns) > 2 else None
    if emotion_col is None:
        print("❌ 감정 컬럼을 찾을 수 없습니다.")
        return

    print(f"📋 감정 컬럼: '{emotion_col}'")

    # 수정 전 감정 분포
    original_emotions = df[emotion_col].dropna().astype(str).str.strip()
    unique_emotions = original_emotions.value_counts()

    print("\n📊 수정 전 감정 분포:")
    for emotion, count in unique_emotions.items():
        marker = "❌" if emotion not in standard_emotions else "✅"
        print(f"  {marker} '{emotion}': {count}개")

    # 오타 수정
    correction_count = 0
    corrections_made = {}

    for typo, correct in typo_corrections.items():
        mask = df[emotion_col].astype(str).str.strip() == typo
        count = mask.sum()
        if count > 0:
            df.loc[mask, emotion_col] = correct
            correction_count += count
            corrections_made[typo] = (correct, count)
            print(f"  🔧 '{typo}' → '{correct}' ({count}개 수정)")

    print(f"\n✅ 총 {correction_count}개 오타 수정 완료")

    # 수정 후 감정 분포
    corrected_emotions = df[emotion_col].dropna().astype(str).str.strip()
    corrected_unique = corrected_emotions.value_counts()

    print("\n📊 수정 후 감정 분포:")
    total_count = 0
    for emotion in standard_emotions:
        count = corrected_unique.get(emotion, 0)
        percentage = (count / len(corrected_emotions)) * 100 if len(corrected_emotions) > 0 else 0
        print(f"  ✅ '{emotion}': {count:,}개 ({percentage:.1f}%)")
        total_count += count

    # 여전히 남은 비표준 감정 확인
    remaining_issues = [e for e in corrected_unique.index if e not in standard_emotions]
    if remaining_issues:
        print(f"\n⚠️ 여전히 남은 비표준 감정: {remaining_issues}")
    else:
        print(f"\n🎉 모든 감정이 표준 7개 감정으로 정리되었습니다!")

    print(f"📈 총 데이터 수: {total_count:,}개")

    # 수정된 파일 저장
    if output_path is None:
        base_name = os.path.splitext(file_path)[0]
        output_path = f"{base_name}_수정본.xlsx"

    try:
        df.to_excel(output_path, index=False, header=True)
        print(f"\n💾 수정된 파일 저장 완료: {output_path}")
    except Exception as e:
        print(f"❌ 파일 저장 오류: {e}")
        return

    return df, corrections_made

# 사용 예시
if __name__ == "__main__":
    file_path = "/content/drive/MyDrive/emotion_project/한국어_연속적_대화_데이터셋.xlsx"

    # 오타 수정 실행
    fixed_df, corrections = fix_emotion_typos(file_path)

    if corrections:
        print("\n📝 수정 요약:")
        for typo, (correct, count) in corrections.items():
            print(f"  - '{typo}' → '{correct}': {count}개")

🔍 감정 오타 수정을 시작합니다...
✅ 파일 로드 완료: 55628개 행
📋 감정 컬럼: '감정'

📊 수정 전 감정 분포:
  ✅ '중립': 43786개
  ✅ '놀람': 4866개
  ✅ '분노': 3628개
  ✅ '슬픔': 1972개
  ✅ '행복': 1030개
  ✅ '혐오': 220개
  ✅ '공포': 98개
  ❌ 'ㅍ': 12개
  ❌ '분': 4개
  ❌ 'ㅈ중립': 1개
  ❌ '중림': 1개
  ❌ '분ㄴ': 1개
  ❌ 'ㄴ중립': 1개
  ❌ '줄': 1개
  🔧 'ㄴ중립' → '중립' (1개 수정)
  🔧 'ㅈ중립' → '중립' (1개 수정)
  🔧 'ㅍ' → '중립' (12개 수정)
  🔧 '분' → '분노' (4개 수정)
  🔧 '분ㄴ' → '분노' (1개 수정)
  🔧 '줄' → '중립' (1개 수정)
  🔧 '중림' → '중립' (1개 수정)

✅ 총 21개 오타 수정 완료

📊 수정 후 감정 분포:
  ✅ '공포': 98개 (0.2%)
  ✅ '놀람': 4,866개 (8.7%)
  ✅ '분노': 3,633개 (6.5%)
  ✅ '슬픔': 1,972개 (3.5%)
  ✅ '중립': 43,802개 (78.8%)
  ✅ '행복': 1,030개 (1.9%)
  ✅ '혐오': 220개 (0.4%)

🎉 모든 감정이 표준 7개 감정으로 정리되었습니다!
📈 총 데이터 수: 55,621개

💾 수정된 파일 저장 완료: /content/drive/MyDrive/emotion_project/한국어_연속적_대화_데이터셋_수정본.xlsx

📝 수정 요약:
  - 'ㄴ중립' → '중립': 1개
  - 'ㅈ중립' → '중립': 1개
  - 'ㅍ' → '중립': 12개
  - '분' → '분노': 4개
  - '분ㄴ' → '분노': 1개
  - '줄' → '중립': 1개
  - '중림' → '중립': 1개
